# Neural Networks in Keras with MNIST

**Time**
- Teaching: 2 hours
- Challenges: 30 minutes

**Questions**
- "How to we load and manipulate input data for deep learning?"
- "How can we use keras to design custom neural networks?"
- "How do we validate our models?"
- "How do we decide neural network architectures that perform best on MNIST?"
- "How do we compare our models and test their ability to generalize?"


**Learning Objectives**
- "Understand and wrangle input to neural networks."
- "The ability to implement, troubleshoot, and modify your own neural networks."

* * * * *

## Installation

If your browser url adress is currently: 

`https://dlab.datahub.berkeley.edu/user/YOURNAME`

Then you are good to go! You are in the dlab's datahub which already has the packages you need to get started along with 2GB of memory.

If you want to work locally on your computer, you should have already followed the installation instructions at <https://github.com/dlab-berkeley/Python-Deep-Learning/blob/main/installation-instructions.md> 

If you haven't yet installed keras and tensorflow locally, I highly suggest using the dlab.datahub for this workshop (link in the installation instructions above).

## Import packages
After installation is complete, we now are able to import the keras library. In order to simplify the syntax we will import the specific functions we need from keras.

Note you can simply import keras, then call each function from the module, for example:

`from tensorflow import keras`

`keras.dataset.mnist()`

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

# Note you may get a warning about CUDA and GPU set up
# You can ignore these for now

We also need to import packages to help us with visualizing and manipulating out data. Remember, this is half the battle!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Keras and MNIST

Keras, Tensorflow, and MNIST, oh my! 

#### Keras
A deep-learning framework developed by google with a user-friendly API built for researchers to quickly prototype models.

#### Tensorflow
A backend engine for Keras.

#### MNIST
A dataset of 60,000 training images and 10,000 test images of handwritten digits. It is often considered the "hello world" of deep learning.

#### Problem Statement

You work for a bank and they need to automate the processing of reading mobile check deposits. 

At they moment they have overworked staff, looking at each photo of the checks and manually inputting the check number, account number, and amount of money. 

Can we make their life easier!?

<img src="https://www.usglobalmail.com/wp-content/uploads/2016/12/check-deposits.png" width="600" />

#### Our Solution

We will build a model that can correctly identify numbers from a picture.

As a first pass, we want to build a model that can take as input an image of a single handwritten digit, and predict the correct target digit.

Let's dive right in and build a neural network model with keras that is able identify handwritten digits! 

## Input Data 

They say 80% of data analysis is data cleaning.

Getting data in the right format is a non-trivial and critical task when  building deep learning models!

### Reading in the data

In [ ]:
def get_mnist_data(subset=True):
    """
    Returns the MNIST dataset as a tuple:
    (x_train, y_train, x_val, y_val, x_test, y_test)
    
    When subset=TRUE:
    Returns only a subset of the mnist dataset.
    Especially important to use if you are on datahub and only have 1-2GB of memory.
    """
    
    if subset:
        N_TRAIN = 5000
        N_VALIDATION = 1000
        N_TEST = 1000
    else:
        N_TRAIN = 48000
        N_VALIDATION = 12000
        N_TEST = 10000
    
    (x_train_and_val, y_train_and_val), (x_test, y_test) = mnist.load_data()
    
    x_train = x_train_and_val[:N_TRAIN,:,:]
    y_train = y_train_and_val[:N_TRAIN]
    
    x_val = x_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION,:,:]
    y_val = y_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION]
    
    x_test = x_test[:N_TEST]
    y_test = y_test[:N_TEST]
    
    return x_train, y_train, x_val, y_val, x_test, y_test
    
    

In [ ]:
# Load the data
# Set subset=False if you want to use the full dataset!
# Note that this will require 2+ GB of memory and will make training take longer

x_train, y_train, x_val, y_val, x_test, y_test = get_mnist_data(subset=True)

### Understanding the data

In [ ]:
def data_summary(data):
    """
    Takes a list of our data partitions and returns the shape.
    """
    
    for i, data_partition in enumerate(data):
        if i == 0:
            print("Training Data")
        elif i == 2:
            print()
            print("Validation Data")
        elif i == 4:
            print()
            print("Testing Data")

        print(f"Shape: {data_partition.shape}")
    
    

In [ ]:
data_summary([x_train, y_train, x_val, y_val, x_test, y_test])

## Challenge 1: Understanding the Input Data

1. Why do we split our data into train, validation, and test sets?
2. What is the shape of our input data partitions?
3. What is the type of the data?

**BONUS:**

4. What is the distribution of the target classes within the data, is it balanced?

### Understanding and visualizing the images

Let's extract just one example from the training data

In [ ]:
one_image = x_train[0, :, :]

Let's look inside and see how it's stored!

In [ ]:
one_image

Not very helpful to see the data in it's raw format 

Instead, let's utilize the shape attribute and matplotlib to help us visualize this image data!

In [ ]:
one_image.shape

In [ ]:
plt.imshow(one_image, cmap=plt.cm.binary);

Which dimension represent rows vs. columns in our image?

Let's find out through a test by grabbing all of the first dimension, and half of the second.

If the image is cutoff column-wise, we know the dimensions are \[ row pixel, column pixel \]

In [ ]:
one_image_first_dimension = one_image[:,0:14]

In [ ]:
plt.imshow(one_image_first_dimension, cmap=plt.cm.binary);

So now we feel solid with our input data format!

The input data has 3 dimensions \[ index for image, pixel row, pixel column \]

### Building a Neural Network in Keras

First we define our neural network architecture.

In [ ]:
first_network = Sequential()

first_network.add(Dense(units = 64,
                        activation= "relu",
                        input_shape=(28*28,)))

Let's review the python code we used to initialize this neural network.

#### Hidden Layer

`first_network = Sequential()`
* This line initializes our model object.

`first_network.add(Dense(...))`
* The add method adds a layer to our model object, specifically a dense layer.

`units = 64`
* Within our dense layer, we specify the number of units or *neurons* we want in our layer.

`activation= "relu"`
* We also specify the activation function we want to use, in this case we use the Rectified Linear Unit (relu).
* This simple activation unit returns x if x > 0, or 0 if x <= 0. 
* A critical component to add non-linearity to our network.

`input_shape=(28*28,)`
* In our first layer must always define the input shape. 
* We pass a single dimensional vector of size 28 \* 28 = 784
* This represents the number of pixels in a single dimension.
* We add the comma to pass in our input shape a tuple.
* Note that subsequent layers we add will already know their input sizes since they are inferred from the previous layer's output size.


In [ ]:
first_network.add(Dense(units = 10,
                        activation = "softmax"))

#### Output Layer
`first_network.add(Dense(...))`
* We add our output layer using the same syntax as our hidden layer.

`units = 10`
* We give this layer 10 units or *neurons* corresponding to the number of classes we are attempting to classify.

`activation = "softmax"`
* We use the softmax activation function for our output layer which gives us back a probability for each class.

We just built our first neural network! It's a simple neural network with a single "hidden" layer, not very *deep*.

We can check out a summary of our model layout with the method `model_object.summary()`

In [ ]:
first_network.summary()

Even this TINY neural network has over 50,000 parameters!

Next we need to give the model:
1. An optimizer strategy
2. A way to calculate loss
3. The metric evaluate during training.

In [ ]:
first_network.compile(optimizer = 'rmsprop', 
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

Lastly, we train the model using the `model_name.fit()` method

If you run the code below.... you would find that the model would fail to train. I do not suggest running it to save memory!

In [ ]:
#history = first_network.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_val, y_val))

Remember that half the battle with model training is understanding what format we need the data in!

We won't go over this in too much detail, but in a snapshot we will:
1. Flatten the pixel dimensions from (28,28) into a single dimension (784)
2. Change pixel data type from integer with pixel values \[0, 225\] to a float values \[0,1\].
3. Expand the y targets from a single dimension with values \[0:9\] to 10 dimensions each representing a target class. 
    - In each row, the target class column will have a value of 1, while the other columns will have a value of 0
    - This is a common technique to reformat categorical targets known as [One Hot Encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/).
 

In [ ]:
def transform_data(xdata, ydata):
    """
    Transforms image data:
        1. Flattens pixel dimensions from 2 -> 1
        2. Scales pixel values between [0,1]
    Transforms target data (ydata):
        - Formats targets as one hot encoded columns
    """
    
    x = {}
    for name, partition in zip(["x_train", "x_val", "x_test"],xdata):
        flatten = partition.reshape((partition.shape[0], 28 * 28))
        scaled = flatten.astype('float32') / 255
        x[name] = scaled
    
    y = {}
    for name, partition in zip(["y_train", "y_val", "y_test"],ydata):
        y[name] = to_categorical(partition)
    
    return x['x_train'], y['y_train'], x['x_val'], y['y_val'], x['x_test'], y['y_test']

In [ ]:
x_train_trans, y_train_trans, x_val_trans, y_val_trans, x_test_trans, y_test_trans = transform_data([x_train, x_val, x_test],
                                                                                                    [y_train, y_val, y_test])

In [ ]:
data_summary([x_train_trans, y_train_trans, x_val_trans, y_val_trans, x_test_trans, y_test_trans])

We have succesfully flattened our input images!

In [ ]:
history = first_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=5, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans), )

### Visualizing the accuracy

In [ ]:
def plot_epoch_accuracy(history_dict):
    """
    Plots the training and validation accuracy of a neural network.
    """
    
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, color = 'navy', alpha = 0.8, label='Training Accuracy')
    plt.plot(epochs, val_acc, color = 'green', label='Validation Accuracy')
    plt.title('Training and validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    return plt.show()

In [ ]:
plot_epoch_accuracy(history.history)

Not too shabby for a *tiny* neural network!



Let's build a second network that has more neurons in the hidden layer (previously: 64, now: 512) and see how effects our classification performance.

In [ ]:
second_network = Sequential()
second_network.add(Dense(512, activation= "relu", input_shape=(28*28,)))
second_network.add(Dense(10, activation="softmax"))

In [ ]:
second_network.compile(optimizer = 'rmsprop', 
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

In [ ]:
history_two = second_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=5, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans))

In [ ]:
plot_epoch_accuracy(history_two.history)

How does adding more neurons effect our training and validation accuracy?

Let's keep training this model for 10 more epochs.

In [ ]:
history_two_more_epochs = second_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=10, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans))

In [ ]:
def combined_epoch_plot(history_dicts):
    """
    Combines two history dictionaries for extended epochs.
    """
    
    combined_history = {key: [] for key in history_dicts[0].keys()}
    
    for hist in history_dicts:
        for key in combined_history.keys():
            combined_history[key].extend(hist[key])
    
    return plot_epoch_accuracy(combined_history)
        
        

In [ ]:
combined_epoch_plot([history_two.history, history_two_more_epochs.history])

Diagnosing our accuracy plot, it looks like even with more epochs, our model is *still* failing to generalize. 

The dunning kruger effect in deep learning.

One solution is to give our add an additional hidden layer to extract more features from the data.

Below we add a second dense layer with the same number of neurons as our first dense layer.

In [ ]:
third_network = Sequential()
third_network.add(Dense(512, activation= "relu", input_shape=(28*28,)))
third_network.add(Dense(512, activation= "relu"))
third_network.add(Dense(10, activation="softmax"))

In [ ]:
third_network.compile(optimizer = 'rmsprop', 
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

In [ ]:
history_two_layers = third_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=10, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans))

In [ ]:
plot_epoch_accuracy(history_two_layers.history)

Our model is still failing to generalize and reach validation accuracy that approaches the training accuracy!

One way we can prevent our model from overfitting to the training data is to add dropout in our hidden layers.

### TLDR for Dropout
Dropout makes our network a bit "forgetful" during training. We get to set a proportion of neurons the network will randomly "forget" or "drop", which we set as a probability in the `model.add(Dropout(probability_here))` function.

This forces our model to generalize, preventing it from overfitting to the training data. 

Check out [Hinton et al. 2012](https://arxiv.org/abs/1207.0580) describing how dropout helps neural network generalization.
- *Fun fact: they also use MNIST to test model generalization :)*

In [ ]:
fourth_network = Sequential()
fourth_network.add(Dense(512, activation= "relu", input_shape=(28*28,)))
fourth_network.add(Dropout(0.5))
fourth_network.add(Dense(512, activation= "relu"))
fourth_network.add(Dropout(0.5))
fourth_network.add(Dense(10, activation="softmax"))

In [ ]:
fourth_network.compile(optimizer = 'rmsprop', 
                     loss = 'categorical_crossentropy',
                     metrics = ['accuracy'])

In [ ]:
history_dropout = fourth_network.fit(x_train_trans, 
                            y_train_trans, 
                            epochs=20, 
                            batch_size=128, 
                            validation_data=(x_val_trans, y_val_trans))

In [ ]:
plot_epoch_accuracy(history_dropout.history)

After implementing dropout, do you think the model is better able to generalize?

## Challenge 2: Build your own neural network

1. Build and compile your own neural network in an object called `my_network`. Feel free to choose your own:
    - Architecture
    - Activation Function
    - Epochs
    
2. Train your model, saving the results to an object called `history_my_network`.

Now that we have several models on hand, let's see how they perform on the holdout test set!

In [ ]:
def get_model_accuracy(model, x_test, y_test):
    """
    Takes a model and a test set of data.
    Returns the accuracy.
    """
    
    score = model.evaluate(x_test, y_test, verbose=0)
    
    accuracy = round(score[1]*100, 1)
    
    return accuracy

In [ ]:
for i, model in enumerate([first_network, second_network, third_network]):
    acc = get_model_accuracy(model, x_test_trans, y_test_trans)
    print(f"Model {i+1} has an accuracy of {acc}%")

## Challenge 3: Evaluate your own model

Use your own model from challenge 2 to evaluate its general performance.

1. Visualize the training and validation accuracy over each epoch.
2. Print the accuracy of your model on the test set.

### Visualize the test results

In [ ]:
def plot_wrong_predictions(model, x_test, y_test):
    """
    Plots 25 incorrectly predicted images.
    """
    
    # Back transform images
    x_images = x_test.reshape(x_test.shape[0], 28, 28)
    
    # Format predictions and targets
    predictions = model.predict(x_test)
    predicted = np.argmax(predictions, axis=1)
    target = np.argmax(y_test, axis = 1)
    
    # Get wrong indices
    wrong_indices = np.where(predicted != target)[0]
    
    fig, axes = plt.subplots(5,5, figsize = (25,25))
    axes = axes.ravel()
    
    for ax, index in zip(axes, wrong_indices[:26]):
        ax.imshow(x_images[index], cmap=plt.cm.binary, interpolation='nearest')
        ax.set_title(f"Predicted {predicted[index]}, Actual is {target[index]}", size = 20)
        ax.axis('off')
    
    return plt.show()

In [ ]:
plot_wrong_predictions(fourth_network, x_test_trans, y_test_trans)

Do you feel our model made reasonable mistakes?

### Saving our models

Training models can be a lengthy and computationally resource-intensive task!

After we train a model, we often want to save it to a file so we can use our trained weights in other applications.

In [ ]:
import os

In [ ]:
def save_models(models, filenames, path = os.path.join("..", "data")):
    """
    Takes a list of model objects and a path to store the models in.
    Saves these models in the path location using the objectname.
    """
    
    # Handle single inputs as one-element lists
    if type(models) is not list:
        models = [models]
    if type(filenames) is not list:
        filenames = [filenames]
    
    assert len(models) == len(filenames), f"There is a mismatch in the number of models: {len(models)} and filenames: {len(filenames)}"
    
    # If the folder to store models doesn't yet exist, make it
    if not os.path.exists(path):
        os.makedirs(path)
    
    for model, file in zip(models, filenames):
        # Make the full filepath
        filepath = os.path.join(path, file)
        
        # Save the model to the filepath
        model.save(filepath)
        print(f"Saved {file} to {filepath}")

The neural networks we have created so far are known as *vanilla neural networks*. In the next notebook we will explore a different neural network architechture known as *convolutional neural networks*.

In [ ]:
save_models([first_network, second_network, third_network, fourth_network],
            ["first_nn", "second_nn", "third_nn", "fourth_nn"])

## Challenge 4: Save your own model

Save your own model `my_network` in the same directory we saved our other models in.